This Notebook is to export the Fourier images to GEE_assest. 

Specifically, we:<br>
1) Loop through 1990-2019 by 3-year intervales;<br>
2) Create the Fourier image using 3-year stack of NDVI/EVI/NDBI, respectively;<br>
3) Export the Fourier image to Assest with name "Amplitude_Phase_{year}"

In [2]:
import ee
import datetime
import os
import itertools
import sys
import collections

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [3]:
ee.Initialize()

# append the parent path into sys-path so wen can import necessary modules
sys.path.append('../')

In [4]:
from BackGround_modules.Class_1_Make_fourier_imgs import Make_Fourier

#### Step_0_Define_Basic_Parameters

In [5]:
# imprt North_China_Plain boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [6]:
# Define the time-range
# just get one_range from every different sensors
year_range = [(f'{i}-01-01',f'{i+2}-12-31') for i in range(1990,2018,3)]

In [7]:
year_range

[('1990-01-01', '1992-12-31'),
 ('1993-01-01', '1995-12-31'),
 ('1996-01-01', '1998-12-31'),
 ('1999-01-01', '2001-12-31'),
 ('2002-01-01', '2004-12-31'),
 ('2005-01-01', '2007-12-31'),
 ('2008-01-01', '2010-12-31'),
 ('2011-01-01', '2013-12-31'),
 ('2014-01-01', '2016-12-31'),
 ('2017-01-01', '2019-12-31')]

#### Step_1_Loop through each year_range to create the harmonic imgs

In [7]:
Fourier_imgs = {}
Amplitude_Phase_imgs = {}

for start,end in year_range:
    
    # define the year_name
    year_name = f'{start[:4]}_{end[:4]}'
    
    # instatiate the instance
    Fourier_instance = Make_Fourier(start_date=start,end_date=end,
                                   area=North_China_Plain)
    
    # make result imgs
    Fourier_instance.Stp_1_Create_hamonic_names()
    Fourier_instance.Stp_2_Add_harmonics()
    Fourier_instance.Stp_3_Harmonic_fit()
    
    # Fourier img
    Fourier_imgs[year_name] = Fourier_instance.harmonicTrendCoefficients
    
    # amplitude_phase img
    Amplitude_Phase_imgs[year_name] = Fourier_instance.Amplitude_Phase_img

Analyzing the images of 1990_1992
Analyzing the images of 1993_1995
Analyzing the images of 1996_1998
Analyzing the images of 1999_2001
Analyzing the images of 2002_2004
Analyzing the images of 2005_2007
Analyzing the images of 2008_2010
Analyzing the images of 2011_2013
Analyzing the images of 2014_2016
Analyzing the images of 2017_2019


##### Step_2_Export the imgs

In [18]:
for year,img in Fourier_imgs.items():

    # construct the export name    
    export_name = f'Amplitude_Phase_{year}'
    
    # here only export the year of 2008-2010, change to full years for fully analysis
    if year == '2008_2010':
        
        target_img = img.clip(North_China_Plain)
        
        # construct the export task
#         task = ee.batch.Export.image.toCloudStorage(image = target_img,
#                                                     description=export_name,
#                                                     bucket='north_china_plain',
#                                                     fileNamePrefix=export_name,
#                                                     region= clip,
#                                                     scale=30,
#                                                     maxPixels=int(1e13),
#                                                     skipEmptyTiles=True)
#         task.start()

        # print out the process
        print(export_name)

Amplitude_Phase_2008_2010


##### Export existing img; 

##### Below is NOT used for analysis of the research!

In [11]:
year = f'2008_2010'

In [12]:
# Fourier img
Fourier = ee.Image(f"users/wang8052664/North_China_Plain/Fourier_imgs/Fourier_img_{year}_harmonic_3")\
                      .select(["NDBI_cos_1","EVI_sin_2","NDVI_sin_2"])

# Landsat_flase_color
False_color = ee.Image(f"users/wang8052664/Cloud_Free_Img/Landsat_cloud_free_{year}").select(['B4','B3','B2'])

#get normalized imgs
NDVI = ee.Image(f'users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDVI').rename('NDVI')
NDBI = ee.Image(f'users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDBI').rename('NDBI')
EVI =  ee.Image(f'users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_EVI').rename('EVI')
Normalized_img = ee.Image([NDVI,NDBI,EVI])

In [15]:
# set the visulization parameters
Landsat_1990_2010 = {"opacity":1,"bands":["B3","B2","B1"],"min":10,"max":39.4,"gamma":1},
Landsat_2011_2019 = {"opacity":1,"bands":["B4","B3","B2"],"min":10,"max":44.8,"gamma":1},
Normalized_1990_2010 = {"opacity":1,"bands":["NDVI","NDBI","EVI"],"min":-15,"max":25,"gamma":1},
Normalized_2011_2013 = {"opacity":1,"bands":["NDBI","NDVI","EVI"],"min":-18.6,"max":40.6,"gamma":1},
Normalized_2014_2019 = {"opacity":1,"bands":["NDVI","NDBI","EVI"],"min":-15,"max":50,"gamma":1},
Fourier_VIS = {"opacity":1,"bands":["NDBI_cos_1","EVI_sin_2","NDVI_sin_2"],"min":-106,"max":158,"gamma":1}

In [20]:
for name,img in zip(['Fourier','False_color','Normalized_img'],
                     [Fourier,False_color,Normalized_img]):
    
    # define the export name
    export_name = f'{name}_{year}'
    
    # export
    task = ee.batch.Export.image.toCloudStorage(image = Normalized_img,
                                                description=export_name,
                                                bucket='north_china_plain',
                                                fileNamePrefix=export_name,
                                                region= North_China_Plain.geometry().bounds(),
                                                scale=30,
                                                maxPixels=int(1e13),
                                                skipEmptyTiles=True)
    task.start()

    # print the process
    print(export_name)

Fourier_2008_2010
False_color_2008_2010
Normalized_img_2008_2010
